In [1]:
# Uncomment only when running first time

#!pip install spotipy
#!pip install bar_chart_race
#!pip install wordcloud


   ---------------------------------------- 0.0/7.8 MB ? eta -:--:--
   ------ --------------------------------- 1.3/7.8 MB 6.7 MB/s eta 0:00:01
   -------- ------------------------------- 1.6/7.8 MB 6.0 MB/s eta 0:00:02
   --------- ------------------------------ 1.8/7.8 MB 3.1 MB/s eta 0:00:02
   --------- ------------------------------ 1.8/7.8 MB 3.1 MB/s eta 0:00:02
   ---------- ----------------------------- 2.1/7.8 MB 2.2 MB/s eta 0:00:03
   ------------- -------------------------- 2.6/7.8 MB 1.9 MB/s eta 0:00:03
   ------------- -------------------------- 2.6/7.8 MB 1.9 MB/s eta 0:00:03
   ---------------- ----------------------- 3.1/7.8 MB 1.8 MB/s eta 0:00:03
   ----------------- ---------------------- 3.4/7.8 MB 1.7 MB/s eta 0:00:03
   ------------------ --------------------- 3.7/7.8 MB 1.7 MB/s eta 0:00:03
   -------------------- ------------------- 3.9/7.8 MB 1.7 MB/s eta 0:00:03
   --------------------- ------------------ 4.2/7.8 MB 1.7 MB/s eta 0:00:03
   ----------------

In [4]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from typing import Union
import pandas as pd
import time 
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
#import plotly.express as px
#import plotly
import bar_chart_race as bcr
from wordcloud import WordCloud
import warnings

# Methods to Download and Format Data from Spotify API

In [5]:
# Change the client id and secret (we will work on this during meeting)
# Steps:
# 1. Log into Spotify for Developers using Spotify Login
# 2. Go to Dashboard and Create New App (You can name it anything and put random info for the other fields)
# 3. Under App Name, there will be the Client ID and a "Show Secret" Button which will show the Client Secret When Clicked

client_id = "8dafe80271644fcdaf587c49f2b1069f"
client_secret = "43a89c7655bd45b7afca764f348efd3e"


client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [6]:
# Gets Spotify Username and Playlist IDs 
# To get Playlist URI, click on "..." -> then on "Share" -> then "Copy link to playlist"
def getTrackIDs(user, playlist_id):
    ids = []
    playlist = sp.user_playlist(user, playlist_id)
    for item in playlist['tracks']['items']:
        track = item['track']
        ids.append(track['id'])
    return ids

In [7]:
# Gets Spotify Singular Song Data From These Locations
# https://developer.spotify.com/documentation/web-api/reference/#/operations/get-track
# https://developer.spotify.com/documentation/web-api/reference/#/operations/get-audio-features
def getTrackFeatures(id):
    meta = sp.track(id)
    features = sp.audio_features(id)

    # Song Meta Data (Most Known Features)
    name = meta['name']
    album = meta['album']['name']
    artist = meta['album']['artists'][0]['name']
    release_date = meta['album']['release_date']
    length = meta['duration_ms']
    popularity = meta['popularity']

    # Song Features (Details)
    acousticness = features[0]['acousticness']
    danceability = features[0]['danceability']
    energy = features[0]['energy']
    instrumentalness = features[0]['instrumentalness']
    liveness = features[0]['liveness']
    loudness = features[0]['loudness']
    speechiness = features[0]['speechiness']
    tempo = features[0]['tempo']
    valence = features[0]['valence']
    time_signature = features[0]['time_signature']
    key = features[0]['key']
    mode = features[0]['mode']
    uri = features[0]['uri']

    track = [name, album, artist, release_date,
             length, popularity, acousticness,
             danceability, energy, instrumentalness, 
             liveness, loudness, speechiness, tempo, 
             valence, time_signature,
             key, mode, uri]
    return track

In [8]:
# Loops Over Track IDs to Get All Songs in Playlist
def loop_playlist(playlist_ids):
    tracks = []
    for i in range(len(playlist_ids)):
        time.sleep(.2)
        track = getTrackFeatures(playlist_ids[i])
        tracks.append(track)
    return tracks

In [9]:
# Turns Data Into Dataframe
def get_spotify_df(tracks):
    df = pd.DataFrame(tracks, columns = ['name', 'album', 'artist', 'release_date',
                                         'length', 'popularity', 'acousticness', 'danceability',
                                         'energy', 'instrumentalness', 'liveness', 'loudness',
                                         'speechiness', 'tempo', 'valence', 'time_signature',
                                         'key', 'mode', 'uri'])
    return df

In [10]:
def get_years(df):
    years = []
    for date in df['release_date'].values:
        if '-' in date:
            years.append(date.split('-')[0])
        else:
            years.append(date)
    df['release_year'] = years
    return df

### Getting Song Data From Playlist

###### Playlist 1

In [9]:
def idsToDf(username, playlistLink):
  playlist = playlistLink
  ids_playlist = getTrackIDs(username, playlist)
  
  ids_playlist_loop = loop_playlist(ids_playlist)
  df_playlist = get_spotify_df(ids_playlist_loop)
  df_playlist = get_years(df_playlist)
  return df_playlist

1.175975795586904


In [20]:
# Converts Data to Dataframe (takes like 3 minutes)

pop_playlist_1 = idsToDf("Chaotic_Duck","https://open.spotify.com/playlist/4HyPpYyFWNaSUsQPYgPBZN?si=16bb42447c9b43e4")
pop_playlist_2 = idsToDf("Chaoitc_Duck", "https://open.spotify.com/playlist/2SG9dz9CnBOFiYLWwZzKwy?si=eecaae1e519b441b")

rock_playlist_1= idsToDf("Nirad", "https://open.spotify.com/playlist/6AuPYCWF6hShXlc30qdFy9?si=3d8ce9b6f3924c27&pt=4437dfc38fede2aa696d85555b7263c2")
rock_playlist_2=idsToDf("Nirad", "https://open.spotify.com/playlist/3qOUFC4ws0bH1JvSayaaV2?si=6d8a7eb3b0c54ca9")


rb_playlist_1 = idsToDf("Ketus","https://open.spotify.com/playlist/5fq0V5zfbxuzGAvoFTWihe?si=c019ee8958b64368")
rb_playlist_2 = idsToDf("Ketus","https://open.spotify.com/playlist/54UV2wXmfwSpi1xV2LiLyG?si=3ce55663372a4ca2")

edm_playlist_1 = idsToDf("Dev Sheth","https://open.spotify.com/playlist/7tIyDAeIc1q71hfPV4szAq?si=069387716477492b")
edm_playlist_2= idsToDf("Dev Sheth","https://open.spotify.com/playlist/2lbSI8yUWilacf0dNMVinR?si=4acecbed444244d4")

rap_hiphop_playlist_1= idsToDf("Rishabh Sabnavis","https://open.spotify.com/playlist/11mtpqmfay3Z0YgG8zlhIP?si=bbad0f78801a4866")
rap_hiphop_playlist_2 = idsToDf("Rishabh Sabnavis","https://open.spotify.com/playlist/6aZTBnigtpPOzbIp21uaTh?si=48e210333b61466f")


frames = [pop_playlist_1,pop_playlist_2,rock_playlist_1,rock_playlist_2,rb_playlist_1,rb_playlist_2,edm_playlist_1,edm_playlist_2, rap_hiphop_playlist_1,rap_hiphop_playlist_2]
merged_df = pd.concat(frames).reset_index(drop=True)
merged_df
merged_df.to_csv(r"C:\Users\amari\file.csv")

Max Retries reached


SpotifyException: http status: 429, code:-1 - /v1/audio-features/?ids=3TKakBgJaTH5INk0WhU4xP:
 Max Retries, reason: too many 429 error responses

In [24]:
merged_df

,name,album,artist,release_date,length,popularity,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,time_signature,key,mode,uri,release_year
0,Skinny Loser,Skinny Loser,Sub Urban,2024-09-20,208658,55,0.301000,0.603,0.881,0.000191,0.3860,-7.987,0.0538,93.012,0.400,4,11,1,spotify:track:25f23uoQ4S8VCQR4KQvvOh,2024
1,Do It Like That,The Name Chapter: FREEFALL,TOMORROW X TOGETHER,2023-10-13,145065,63,0.010300,0.776,0.920,0.000000,0.3250,-3.246,0.0548,119.989,0.842,4,5,0,spotify:track:0XM1XnOYv8BkFPFVCU8ZjX,2023
2,LOOK SO PRETTY,LOOK SO PRETTY,Gun Boi Kaz,2024-09-27,153600,37,0.232000,0.724,0.501,0.000000,0.0899,-12.024,0.0653,125.040,0.542,4,0,1,spotify:track:2DhwBiwPbQXoFIjNzeDNXg,2024
3,glhf<3,glhf<3,MICO,2024-07-19,176326,51,0.016900,0.722,0.813,0.000000,0.0992,-6.906,0.0609,147.009,0.674,4,0,1,spotify:track:55SQgo2TTd0cjiWGzY2Q9A,2024
4,MADE UP YOUR MIND,MADE UP YOUR MIND,Gun Boi Kaz,2024-08-23,164307,42,0.125000,0.812,0.353,0.000139,0.1060,-12.023,0.0775,129.956,0.953,4,9,1,spotify:track:2kTQ8rRufN5BjtfVh2O7VU,2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,POSTED IN THE DVRK,DRAIN THE SWAMP,Lil Darkie,2018-12-25,155000,43,0.561000,0.715,0.932,0.000000,0.9390,0.112,0.3970,156.143,0.860,4,11,1,spotify:track:1n5VXJ8VamAUhqMXHgn279,2018
596,Not Like Us,Not Like Us,Kendrick Lamar,2024-05-04,274192,91,0.010700,0.898,0.472,0.000000,0.1410,-7.001,0.0776,101.061,0.214,4,1,1,spotify:track:6AI3ezQ4o3HUoP6Dhudph3,2024
597,The Motto,Take Care (Deluxe),Drake,2011-11-15,181573,72,0.000107,0.766,0.442,0.000061,0.1110,-8.558,0.3560,201.800,0.390,4,1,1,spotify:track:4Kz4RdRCceaA9VgTqBhBfa,2011
598,You (feat. Travis Scott),Life of a DON,Don Toliver,2021-10-08,213637,71,0.409000,0.782,0.528,0.000079,0.0999,-6.035,0.0532,117.977,0.389,4,1,0,spotify:track:4qS2KPWvsQzLvRa9oCHw41,2021


In [25]:
# Converts Data to Dataframe
merged_df.to_csv(r"C:\Users\amari\file.csv",index=False)


In [11]:
# Saves DataFrame to csv File


###### Playlist 2

In [12]:
# Change Playlist URI and Username
'''
mellow_playlist = "https://open.spotify.com/playlist/4DI82TTpna4sIMDJWLHZQu?si=5b4df9b436ca4678"
ids_mellow = getTrackIDs("Chaotic_Duck", mellow_playlist)
time_start = time.time()
ids_mellow_loop = loop_playlist(ids_mellow)
time_end = time.time()
# Prints out time taken to loop through playlist in minutes (as playlist gets longer, might take closer to 4-5 minutes)
print((time_end - time_start)/60)
'''

1.3952751954396565


ReadTimeout: HTTPSConnectionPool(host='api.spotify.com', port=443): Read timed out. (read timeout=5)

###### Playlist 3

In [13]:
# Change Playlist URI and Username
'''
study_playlist = "https://open.spotify.com/playlist/3aJ4UQWGTeeCLti4KxmJ7D?si=c250e256405540b1"
ids_study = getTrackIDs("Chaotic_Duck", study_playlist)
time_start = time.time()
ids_study_loop = loop_playlist(ids_study)
time_end = time.time()
# Prints out time taken to loop through playlist in minutes (as playlist gets longer, might take closer to 4-5 minutes)
print((time_end - time_start)/60)
'''

0.6649918794631958


In [14]:
# Converts Data to Dataframe
'''
df_study = get_spotify_df(ids_study_loop)
df_study = get_years(df_study)
df_study
'''

,name,album,artist,release_date,length,popularity,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,time_signature,key,mode,uri,release_year
0,Double Knot,Double Knot,Stray Kids,2019-10-09,189771,0,0.048600,0.716,0.814,0.000000,0.607,-2.889,0.3500,146.925,0.575,4,5,0,spotify:track:7vZseKiQOIuFxxfi84302k,2019
1,EMPIRE,EMPIRE,WENGIE,2019-10-18,182666,40,0.037500,0.730,0.830,0.000000,0.306,-3.001,0.0629,90.030,0.533,4,5,1,spotify:track:6DfcCK3nYr7Fxl2af3iqRB,2019
2,Rush Hour,NO LIMIT,MONSTA X,2021-11-19,202840,0,0.036700,0.772,0.889,0.000000,0.685,-2.496,0.2220,112.866,0.705,4,5,0,spotify:track:1Goejji6m1d0s0cv4OnL7f,2021
3,Scars,Scars,Stray Kids,2021-10-06,200546,55,0.030300,0.542,0.671,0.000000,0.123,-4.109,0.0840,79.997,0.451,4,0,1,spotify:track:4C9Zhjq5ux9ieiDWYfFcfP,2021
4,Tamed-Dashed,DIMENSION : DILEMMA,ENHYPEN,2021-10-12,196170,63,0.000253,0.568,0.809,0.000033,0.151,-3.869,0.0517,155.029,0.709,4,11,0,spotify:track:1zoyteFQmeUUqyOl2Xznpy,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Happy Death Day,Happy Death Day,Xdinary Heroes,2021-12-06,220396,0,0.024300,0.677,0.841,0.000000,0.160,-2.970,0.0813,130.024,0.418,4,9,0,spotify:track:3a4VCqofc7Wf7cqfZYu2Ad,2021
96,Favorite (Vampire),Favorite - The 3rd Album Repackage,NCT 127,2021-10-25,215586,55,0.000831,0.561,0.758,0.000003,0.150,-3.870,0.0478,149.944,0.178,4,5,0,spotify:track:2R8MZR1RCP4cIJITHDPRbY,2021
97,AYAYA,DISHARMONY : BREAK OUT,P1Harmony,2021-04-20,150533,47,0.127000,0.740,0.798,0.000000,0.114,-3.865,0.0733,129.953,0.765,4,9,0,spotify:track:2nIQrNY2PDVL9elv6Qg1Qc,2021
98,End It,DISHARMONY : BREAK OUT,P1Harmony,2021-04-20,212973,41,0.050200,0.435,0.898,0.000000,0.437,-2.833,0.2370,173.988,0.589,4,10,0,spotify:track:6xZLTFK96jeEFkkldwKMsA,2021


# Data Visualization

###### Playlist 1

In [32]:

#merged_df[['name','artist','album','release_date']].head()

,name,artist,album,release_date
0,Dying In A Hot Tub,Palaye Royale,Boom Boom Room (Side B),2018-09-28
1,Lonely,Palaye Royale,The Bastards,2020-05-29
2,Choke,I DONT KNOW HOW BUT THEY FOUND ME,Choke,2018-08-24
3,Little Bastards,Palaye Royale,The Bastards,2020-05-29
4,Mr. Doctor Man,Palaye Royale,Boom Boom Room (Side A),2016-06-24


In [33]:
'''
df_sorted = merged_df[['artist','name']].groupby('artist').count().sort_values('name',ascending = False).reset_index()
df_sorted.columns = ['Artist','Count']
df_sorted.head(100)'''

,Artist,Count
0,Palaye Royale,41
1,ENHYPEN,31
2,KIM WOO SEOK,18
3,The Used,17
4,ZEROBASEONE,11
5,EVNNE,11
6,The Faim,7
7,ONE PACT,6
8,AB6IX,5
9,8TURN,5


In [24]:
'''num_artists = len(df_workout[['artist']].groupby('artist').count())
print(f'You had {num_artists} artists in your playlist')'''

You had 18 artists in your playlist


###### Playlist 2

In [26]:
#df_mellow[['name','artist','album','release_date']].head()

,name,artist,album,release_date
0,WHISTLE,AB6IX,THE FUTURE IS OURS: FOUND,2024-01-22
1,GRAB ME,AB6IX,THE FUTURE IS OURS: FOUND,2024-01-22
2,TRAVELER,AB6IX,THE FUTURE IS OURS: FOUND,2024-01-22
3,ALL NIGHT,AB6IX,THE FUTURE IS OURS: FOUND,2024-01-22
4,ILY (I LOVE YOU),AB6IX,THE FUTURE IS OURS: FOUND,2024-01-22


In [27]:
'''df_sorted_mellow = df_mellow[['artist','name']].groupby('artist').count().sort_values('name',ascending = False).reset_index()
df_sorted.columns = ['Artist','Count']
df_sorted.head(10)'''

,Artist,Count
0,ENHYPEN,31
1,KIM WOO SEOK,18
2,EVNNE,11
3,ZEROBASEONE,11
4,ONE PACT,6
5,8TURN,5
6,AB6IX,5
7,ALL(H)OURS,4
8,DXMON,4
9,NCT WISH,2


In [28]:
'''num_artists = len(df_mellow[['artist']].groupby('artist').count())
print(f'You had {num_artists} artists in your playlist')'''

You had 13 artists in your playlist


###### Playlist 3

In [29]:
#df_study[['name','artist','album','release_date']].head()

,name,artist,album,release_date
0,Double Knot,Stray Kids,Double Knot,2019-10-09
1,EMPIRE,WENGIE,EMPIRE,2019-10-18
2,Rush Hour,MONSTA X,NO LIMIT,2021-11-19
3,Scars,Stray Kids,Scars,2021-10-06
4,Tamed-Dashed,ENHYPEN,DIMENSION : DILEMMA,2021-10-12


In [30]:
'''df_sorted_study = df_study[['artist','name']].groupby('artist').count().sort_values('name',ascending = False).reset_index()
df_sorted.columns = ['Artist','Count']
df_sorted.head(10)'''

,Artist,Count
0,TOMORROW X TOGETHER,12
1,ENHYPEN,8
2,Stray Kids,8
3,THE BOYZ,7
4,P1Harmony,6
5,MONSTA X,5
6,ATEEZ,4
7,EXO,3
8,NCT 127,3
9,Dreamcatcher,3


In [31]:
'''num_artists = len(df_study[['artist']].groupby('artist').count())
print(f'You had {num_artists} artists in your playlist')'''

You had 42 artists in your playlist


### Comparing Artists in Playlists

In [ ]:
#plt.rcParam['figure.figsize'] = (24,12)

In [35]:

'''for df, playlist, count in zip(
    [df_sorted, df_sorted_mellow, df_sorted_study],
    ['Workout', 'Mellow', 'Study'],
    ['count_workout', 'count_mellow', 'count_study']):
    df['artist'] = df.iloc[:,0]
    df['playlist'] = [playlist] * len(df)
    df['count'] = df[count]
    df = df[['artist', 'playlist', 'count']]

bar_chart_race_df = pd.concat([df_sorted, df_sorted_mellow, df_sorted_study], ignore_index=True, axis=0, sort=False)[['artist', 'playlist', 'count']]

df_bcr = pd.DataFrame(index=['Workout', 'Mellow', 'Study'])
for artist,year,count in zip(bar_chart_race_df['artist'],bar_chart_race_df['playlist'],bar_chart_race_df['count']):
    df_bcr.loc[year, artist] = count
df_bcr = df_bcr.fillna(0)
for c in df_bcr.columns:
    df_bcr[c] = df_bcr[c].cumsum()
df_bcr '''

IndentationError: expected an indented block after 'for' statement on line 1 (1572067310.py, line 2)